In [1]:
import sys
sys.path.append('/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage')

from RegionalExtremesPackage.main import parser_arguments, regional_extremes_method, local_extremes_method, regional_extremes_minicube


/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage


In [2]:
import numpy as np

from dask_jobqueue import SLURMCluster
from dask.distributed import Client as daskClient

In [3]:
cluster = SLURMCluster(
    queue='work',                  # Specify the SLURM queue
    processes=1,                     # Number of processes per job
    cores=1,                          # Number of cores per job
    memory='1GB',                    # Memory per job
    walltime='03:00:00',              # Job duration (hh:mm:ss)
    job_script_prologue=[
        'module load BGC-easybuilded',
        'module load  GCC',
    ]
)

# Scale up the number of workers
# cluster.scale(jobs=8)  # Adjust the number of jobs/workers
cluster.adapt(minimum=0, maximum=20)

# Create a Dask client that connects to the cluster
client = daskClient(cluster)

# Check cluster status
cluster

/Net/Groups/BGI/scratch/crobin/miniconda3/envs/ExtremesEnv2/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44701 instead
  warnings.warn(


Dashboard: http://10.0.30.56:44701/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.30.56:34825,Workers: 0
Dashboard: http://10.0.30.56:44701/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
import sys
import argparse

# Remove Jupyter's arguments before parsing your own
sys.argv = sys.argv[:1]


args = parser_arguments().parse_args()
args

Namespace(id=None, name=None, index='pei_180', compute_variance=False, region='globe', time_resolution=5, n_components=3, n_samples=100, n_bins=25, kernel_pca=False, saving_path=None, path_load_experiment=None, method='regional')

In [5]:
import numpy as np 

# Remove Jupyter's arguments before parsing your own
sys.argv = sys.argv[:1]
args = parser_arguments().parse_args()
args.name = "deep_extreme_HR"
args.index = "EVI_EN"
args.k_pca = False
args.n_samples = 1000
args.n_components = 3
args.n_bins = 50
args.compute_variance = False
args.method = "regional"
args.start_year = 2000
args.is_generic_xarray_dataset = False
args.path_load_experiment = "/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/experiments/2024-11-25_18:41:43_deep_extreme_HR"
LOWER_QUANTILES_LEVEL = np.array([0.01, 0.025, 0.05])
UPPER_QUANTILES_LEVEL = np.array([0.95, 0.975, 0.99])
if args.method == "regional":
    # Apply the regional extremes method
    # regional_extremes_method(args, (LOWER_QUANTILES_LEVEL, UPPER_QUANTILES_LEVEL))
    regional_extremes_minicube(args, (LOWER_QUANTILES_LEVEL, UPPER_QUANTILES_LEVEL))
elif args.method == "local":
    # Apply the uniform threshold method
    local_extremes_method(args, (LOWER_QUANTILES_LEVEL, UPPER_QUANTILES_LEVEL))
elif args.method == "global":
    raise NotImplementedError("the global method is not yet implemented.")



[2024-12-06 14:44:48] Loading of the model path: /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/experiments/2024-11-25_18:41:43_deep_extreme_HR
[2024-12-06 14:45:11] Projection loaded from /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/experiments/2024-11-25_18:41:43_deep_extreme_HR/EVI_EN/pca_projection_3.zarr
[2024-12-06 14:45:11] Limits bins loaded.
[2024-12-06 14:45:11] Bins loaded.
[2024-12-06 14:45:11] The file /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/experiments/2024-11-25_18:41:43_deep_extreme_HR/EVI_EN/thresholds.zarr not found.
[2024-12-06 14:45:11] start of the preprocess


/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/datahandler.py:579: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(metadata_file, delimiter=",")[
/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/datahandler.py:579: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(metadata_file, delimiter=",")[
/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage/datahandler.py:579: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(metadata_file, delimiter=",")[


AttributeError: 'DataArray' object has no attribute 'data_id'

In [ ]:
client.close()
cluster.close()